In [8]:
import openai
import os
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from openai import OpenAI

In [6]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [9]:
client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': 'Is OpenAI free to use?'}]
)

ChatCompletion(id='chatcmpl-BfoeBpdkWw5wnEIzpd72QCtVS4yYX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='OpenAI offers both free and paid options for accessing its models and tools. For instance:\n\n1. **ChatGPT**:\n   - There is a free tier available. Anyone can sign up and use it, though there might be limitations on usage and access compared to the paid version.\n   - OpenAI also offers a subscription plan called "ChatGPT Plus" for a monthly fee. This provides benefits such as faster response times, priority access during peak times, and access to newer models.\n\n2. **API Access**:\n   - OpenAI\'s API has a pricing model based on usage. Developers and businesses can sign up and use the API, but beyond initial credits, usage incurs costs based on the amount of processing and data involved.\n\n3. **Research and Education**:\n   - Occasionally, OpenAI offers grants or credits for research and educational purposes, allowing certai

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context = ''
    for doc in search_results:
        # context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        context = context + context_template.format(question=doc['question'], text=doc['text']).strip() + "\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [4]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
es_client.info()

ObjectApiResponse({'name': 'd5d74604fe1c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ao2X3hg6QcOt9GJlKO003w', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name="hw-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hw-questions'})

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [17]:
q3 = "How do execute a command on a Kubernetes pod?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": q3,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)
response

ObjectApiResponse({'took': 19, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'hw-questions', '_id': '10RrUpcBGkD8Chh5a2PJ', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'hw-questions', '_id': 'ZkRrUpcBGkD8Chh5bWT9', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-das

In [18]:
q4 = "How do copy a file to a Docker container?"
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": q4,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][-1]

{'_index': 'hw-questions',
 '_id': '90RrUpcBGkD8Chh5bGM5',
 '_score': 59.812744,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

In [20]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs
    
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context = ''
    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']).strip() + "\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [21]:
search_result = elastic_search(q4)
prompt = build_prompt(q4, search_result)

In [22]:
len(prompt)

1446

In [23]:
import tiktoken

In [24]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [25]:
encoded_prompt = encoding.encode(prompt)

In [26]:
len(encoded_prompt)

320